In [ ]:
import time 

import requests

import bs4

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim
import geopy.geocoders

import json

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

from bs4 import BeautifulSoup

Downloading data from Wikipedia - list of Warsaw's districts

In [ ]:
data = requests.get('https://en.wikipedia.org/wiki/Districts_of_Warsaw').text
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
neighborhood = []
population = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        neighborhood.append(cells[0].text)
        population.append(cells[1].text) # remove the new line char from neighborhood cell

In [ ]:
warsaw_df = pd.DataFrame(list(zip(neighborhood, population)), 
                          columns = ['District', 'Population'])

warsaw_df['Latitude'] = 0
warsaw_df['Longitude'] = 0
warsaw_df.drop(warsaw_df.tail(1).index,inplace=True)
warsaw_df

Add geographical coordinates to each district

In [ ]:
for i in range(len(warsaw_df)):
    address = warsaw_df.iloc[i, 0] + ', Warsaw'
    geolocator = Nominatim(user_agent='warsaw')
    location = geolocator.geocode(address)
    if location == None:
        continue
    latitude = location.latitude
    longitude = location.longitude
    warsaw_df.iloc[i,2] = latitude
    warsaw_df.iloc[i,3] = longitude
    
warsaw_df

Get coordinates of Warsaw

In [ ]:
address = 'Warsaw, Poland'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Warsaw are {}, {}.'.format(latitude, longitude))

Map of Warsaw with districts

In [ ]:
map_warsaw = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(warsaw_df['Latitude'], warsaw_df['Longitude'], 
                                           warsaw_df['District']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_warsaw)  
    
map_warsaw

In [ ]:
CLIENT_ID = 'xxx' # your Foursquare ID
CLIENT_SECRET = 'xxx' # your Foursquare Secret
VERSION = '20190928' # Foursquare API version

Use of Foursquare API to get restaurants data

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=3000, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=Food'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
from requests_oauthlib import OAuth2Session

In [ ]:
warsaw_nearby_venues = getNearbyVenues(warsaw_df.District,
                            warsaw_df.Latitude,
                            warsaw_df.Longitude)

In [ ]:
print(warsaw_nearby_venues.shape)
warsaw_nearby_venues.head()

In [ ]:
arsaw_nearby_venues[warsaw_nearby_venues['District'] == 'Mokotów']

In [ ]:
warsaw_nearby_venues.groupby('District').count()

In [ ]:
print('There are {} uniques categories.'.format(len(warsaw_nearby_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
warsaw_onehot = pd.get_dummies(warsaw_nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
warsaw_onehot['District'] = warsaw_nearby_venues['District'] 

# # move neighborhood column to the first column
nindex = list(warsaw_onehot.columns).index("District")

cols = warsaw_onehot.columns

cols = list(cols)

cols_new = list() 
cols_new.append(cols[list(warsaw_onehot.columns).index("District")])
cols_new.extend(cols[0:list(warsaw_onehot.columns).index("District")])
cols_new.extend(cols[list(warsaw_onehot.columns).index("District")+1:])

warsaw_onehot = warsaw_onehot[cols_new]

warsaw_onehot.head()

In [ ]:
warsaw_onehot.shape

In [ ]:
warsaw_grouped = warsaw_onehot.groupby('District').mean().reset_index()
warsaw_grouped

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['District'] = warsaw_grouped['District']

for ind in np.arange(warsaw_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(warsaw_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Clustering
KMeans clustering

In [ ]:
# set number of clusters
kclusters = 5

warsaw_grouped_clustering = warsaw_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(warsaw_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

warsaw_merged = warsaw_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
warsaw_merged = warsaw_merged.join(neighborhoods_venues_sorted.set_index('District'), on='District')

warsaw_merged['Cluster Labels'] =  warsaw_merged['Cluster Labels'].astype("int")
warsaw_merged.head()

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(warsaw_merged['Latitude'], warsaw_merged['Longitude'], 
                                  warsaw_merged['District'], warsaw_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
warsaw_merged[warsaw_merged['Cluster Labels'] == 0]

In [ ]:

warsaw_merged[warsaw_merged['Cluster Labels'] == 1]

In [ ]:
warsaw_merged[warsaw_merged['Cluster Labels'] == 2]

In [ ]:
warsaw_merged[warsaw_merged['Cluster Labels'] == 3]

In [ ]:
warsaw_merged[warsaw_merged['Cluster Labels'] == 4]
